# Coursera Data Science Capstone Week3

## FIRST SUBMISSION

## Creating the Dataframe

In [23]:
import pandas
from IPython.display import display_html

In [24]:
url1 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [25]:
df1 = pandas.read_html(url1,header=0)[0]

In [26]:
#Removing 'Borough' = 'Not assigned'
df2 = df1.drop(df1[df1['Borough']=="Not assigned"].index)
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [27]:
def Combine_Neigh(x):
    return pandas.Series(dict(Neighbourhood=','.join(x['Neighbourhood'])))

In [28]:
#Combining Neighbourhood for (Postcode,Borough) group
df3 = df2.groupby(['Postcode','Borough'],sort=False).apply(Combine_Neigh).reset_index()
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [29]:
#Replacing 'Not assigned' Neighbourhood values with Borough values
for i in range(len(df3)):
    if df3['Neighbourhood'][i] == "Not assigned":
        df3['Neighbourhood'][i] = df3['Borough'][i]
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [30]:
df3.shape

(103, 3)

## SECOND SUBMISSION

In [31]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [32]:
import geocoder # import geocoder
#Geocoder not working. Infinite loop.

In [34]:
#Create dataframe of Geospatial Coordinates
df4 = pandas.read_csv('C:/Users/User/Downloads/Geospatial_Coordinates.csv')
df4.rename(columns={'Postal Code':'Postcode'},inplace=True)
df4.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df5 = pandas.merge(df3,df4,on='Postcode')
df5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [36]:
df5.shape

(103, 5)